# Dataset Plot Accessor Demo (`.fxplot`)

This notebook demonstrates the new `.fxplot` accessor for `xr.Dataset` objects.
It provides convenient Plotly Express plotting methods with smart auto-faceting and coloring.

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr

import flixopt as fx

fx.__version__

## Create Sample Data

Let's create a multi-dimensional dataset to demonstrate the plotting capabilities.

In [ ]:
# Simple time-series dataset
np.random.seed(42)
time = pd.date_range('2024-01-01', periods=24, freq='h')

ds_simple = xr.Dataset(
    {
        'Solar': (['time'], np.maximum(0, np.sin(np.linspace(0, 2 * np.pi, 24)) * 50 + np.random.randn(24) * 5)),
        'Wind': (['time'], np.abs(np.random.randn(24) * 20 + 30)),
        'Demand': (['time'], np.abs(np.sin(np.linspace(0, 2 * np.pi, 24) + 1) * 40 + 50 + np.random.randn(24) * 5)),
    },
    coords={'time': time},
)

ds_simple

## Line Plot

In [ ]:
ds_simple.fxplot.line(title='Energy Generation & Demand')

## Stacked Bar Chart

In [ ]:
ds_simple[['Solar', 'Wind']].fxplot.stacked_bar(title='Renewable Generation')

## Area Chart

In [ ]:
ds_simple[['Solar', 'Wind']].fxplot.area(title='Stacked Area - Generation')

## Grouped Bar Chart

In [ ]:
ds_simple.fxplot.bar(title='Grouped Bar Chart')

## Heatmap

In [ ]:
# Create 2D data for heatmap
ds_heatmap = xr.Dataset(
    {
        'temperature': (['day', 'hour'], np.random.randn(7, 24) * 5 + 20),
    },
    coords={
        'day': pd.date_range('2024-01-01', periods=7, freq='D'),
        'hour': range(24),
    },
)

ds_heatmap.fxplot.heatmap('temperature', title='Temperature Heatmap')

## Automatic Faceting & Animation

Extra dimensions are **automatically** assigned to `facet_col`, `facet_row`, and `animation_frame` based on CONFIG priority. Just call the plot method - no configuration needed!

In [ ]:
# Dataset with scenario AND period dimensions
ds_multi = xr.Dataset(
    {
        'Solar': (['time', 'scenario', 'period'], np.random.rand(24, 2, 3) * 50),
        'Wind': (['time', 'scenario', 'period'], np.random.rand(24, 2, 3) * 40 + 20),
    },
    coords={
        'time': time,
        'scenario': ['base', 'high'],
        'period': ['winter', 'spring', 'summer'],
    },
)

ds_multi

In [ ]:
# Just call .line() - dimensions are auto-assigned to facet_col, facet_row, animation_frame
ds_multi.fxplot.line(title='Auto-Faceted: Just Works!')

In [ ]:
# Same for stacked bar - auto-assigns period to facet_col, scenario to animation
ds_multi.fxplot.stacked_bar(title='Stacked Bar: Also Just Works!')

## Customizing Facets & Animation

Override auto-assignment when needed. Use `None` to disable a slot entirely.

In [ ]:
# Swap: put scenario in facet_col, period in animation
ds_multi.fxplot.line(facet_col='scenario', animation_frame='period', title='Swapped: Scenario in Columns')

In [ ]:
# Use both row and column facets - no animation
ds_multi.fxplot.area(facet_col='scenario', facet_row='period', animation_frame=None, title='Grid: Period × Scenario')

In [ ]:
# Or reduce dimensions with .sel() for a simpler plot
ds_multi.sel(scenario='base', period='summer').fxplot.line(title='Single Slice: No Faceting Needed')

## Custom Colors

In [ ]:
# Using a colorscale name
ds_simple.fxplot.line(colors='viridis', title='With Viridis Colorscale')

In [ ]:
# Using explicit color mapping
ds_simple.fxplot.stacked_bar(
    colors={'Solar': 'gold', 'Wind': 'skyblue', 'Demand': 'salmon'}, title='With Custom Colors'
)

## Chaining with Plotly Methods

Since all methods return `go.Figure`, you can chain Plotly's update methods.

In [ ]:
(
    ds_simple.fxplot.line(title='Customized Plot')
    .update_layout(xaxis_title='Time of Day', yaxis_title='Power (MW)', legend_title='Source', template='plotly_white')
    .update_traces(line_width=2)
)

## Pre-filtering with xarray

Filter data using xarray methods before plotting.

In [ ]:
# Select specific time range
ds_simple.sel(time=slice('2024-01-01 06:00', '2024-01-01 18:00')).fxplot.line(title='Daytime Only')

In [ ]:
# Select specific variables
ds_simple[['Solar', 'Wind']].fxplot.area(title='Renewables Only')

## DataArray Accessor

The `.fxplot` accessor also works on `xr.DataArray` objects directly.

In [ ]:
# Create a DataArray
da = xr.DataArray(
    np.random.randn(24, 7) * 5 + 20,
    dims=['time', 'day'],
    coords={
        'time': pd.date_range('2024-01-01', periods=24, freq='h'),
        'day': pd.date_range('2024-01-01', periods=7, freq='D'),
    },
    name='temperature',
)

# Heatmap directly from DataArray
da.fxplot.heatmap(title='DataArray Heatmap')

In [ ]:
# Line plot from DataArray (converts to Dataset internally)
da_1d = xr.DataArray(
    np.sin(np.linspace(0, 4 * np.pi, 100)) * 50,
    dims=['time'],
    coords={'time': pd.date_range('2024-01-01', periods=100, freq='h')},
    name='signal',
)
da_1d.fxplot.line(title='DataArray Line Plot')

## Axis Labels

Use `xlabel` and `ylabel` parameters to customize axis labels.

In [ ]:
ds_simple.fxplot.line(title='Generation with Custom Axis Labels', xlabel='Time of Day', ylabel='Power [MW]')

## Scatter Plot

Plot two variables against each other.

In [ ]:
# Basic scatter plot
ds_simple.fxplot.scatter(
    x='Solar', y='Demand', title='Solar vs Demand Correlation', xlabel='Solar Generation [MW]', ylabel='Demand [MW]'
)

In [ ]:
# Scatter with faceting by period, for one scenario
ds_multi.sel(scenario='high').fxplot.scatter(
    x='Solar', y='Wind', facet_col='period', title='Solar vs Wind by Period (High Scenario)'
)

## Pie Chart

Aggregate data to at most 1D per variable. Scalar data creates a single pie; 1D data creates faceted pies.

In [ ]:
# Single pie from scalar values (sum over time)
ds_simple[['Solar', 'Wind']].sum('time').fxplot.pie(
    title='Total Generation by Source', colors={'Solar': 'gold', 'Wind': 'skyblue'}
)

In [ ]:
# Faceted pie - one pie per period (for one scenario)
ds_multi.sum('time').fxplot.pie(
    facet_col='period',
    title='Generation by Source per Period (Base Scenario)',
    colors={'Solar': 'gold', 'Wind': 'skyblue'},
)

## Duration Curve

Sort values along the time dimension to create a duration curve. Useful for analyzing capacity utilization.

In [ ]:
# Duration curve with normalized x-axis (percentage)
ds_simple.fxplot.duration_curve(title='Generation Duration Curves')

In [ ]:
# Duration curve with absolute timesteps on x-axis
ds_simple.fxplot.duration_curve(normalize=False, title='Duration Curves (Timesteps)')

In [ ]:
# Duration curve with faceting by period (for one scenario)
ds_multi.sel(scenario='base').fxplot.duration_curve(facet_col='period', title='Duration Curves by Period')

## Line Shape Configuration

The default line shape is controlled by `CONFIG.Plotting.default_line_shape` (default: `'hv'` for step plots).
Override per-plot with the `line_shape` parameter. Options: `'linear'`, `'hv'`, `'vh'`, `'hvh'`, `'vhv'`, `'spline'`.

In [ ]:
# Default step plot (hv)
ds_simple[['Solar']].fxplot.line(title='Default Step Plot (hv)')

In [ ]:
# Override to linear interpolation
ds_simple[['Solar']].fxplot.line(line_shape='linear', title='Linear Interpolation')